In [1]:
import numpy as np
from ikpy.chain import Chain
from ikpy.link import URDFLink
from scipy.spatial.transform import Rotation as R
from ikpy.utils import geometry

joint_limits_min = np.array([-166, -101, -166, -176, -166, -1, -166])
joint_limits_max = np.array([166, 101, 166, -4, 166, 215, 166])
lims_min_safe = joint_limits_min + 1
lims_max_safe = joint_limits_max -1
joint_limits_rad_min = np.radians(lims_min_safe) 
joint_limits_rad_max = np.radians(lims_max_safe) 

franka_chain = Chain.from_urdf_file(
    "franka.urdf",
    base_elements=["panda_panda_link0"],
    last_link_vector=np.array([0, 0, 0.045]),
    active_links_mask = [False, True, True, True, True, True, True, True, False, False, False, False]
)
#set bounds
for i in range(len(joint_limits_rad_min)):
    franka_chain.links[i + 1].bounds = (joint_limits_rad_min[i], joint_limits_rad_max[i])


In [2]:
    start_p = [-0.1, -0.2, 0.85]
    start_o = [0, 0, -1]
    prev_angle =  [1, -0.38066329, 0.26839613, -2, 0.77729231, 2, 0.00391651, 0.79768890]
    full_prev_angle = franka_chain.active_to_full(prev_angle, np.zeros(len(franka_chain.links)))
    #joints_start = franka_chain.inverse_kinematics(start_p, initial_position=full_prev_angle)

    quaternation = [ 6.123234e-17,  6.123234e-17,  1.000000e+00, -6.123234e-17]
    rotation_matrix = R.from_quat(quaternation).as_matrix()
    
    start_frame = np.eye(4)
    start_frame[:3, 3] = start_p
    start_frame[:3, :3] = rotation_matrix
    
    joints_start = franka_chain.inverse_kinematics_frame(start_frame, initial_position=full_prev_angle)
    print(joints_start)

[ 0.          1.38123689 -1.12529545 -0.34540107 -1.23094605  1.20562574
  0.64190095 -0.35178801  0.          0.          0.          0.        ]


In [3]:
#!/usr/bin/env python
"""Example demonstrating collision detection and shortest distance queries."""

import time
import numpy as np
import pybullet as pyb
import pybullet_data
import pyb_utils

TIMESTEP = 1.0 / 60
all_x, all_y, all_z = [],[],[]
def load_environment(client_id, difficulty):
    if difficulty == 1:
        pyb.setTimeStep(TIMESTEP, physicsClientId=client_id)
        pyb.setAdditionalSearchPath(
            pybullet_data.getDataPath(), physicsClientId=client_id
        )
        
        # ground plane
        ground_id = pyb.loadURDF(
            "plane.urdf", [0, 0, 0], useFixedBase=True, physicsClientId=client_id
        )
    
        # KUKA iiwa robot arm (franka)
        franka_id = pyb.loadURDF(
            "franka.urdf",
            [0, 0, 0],
            useFixedBase=True,
            physicsClientId=client_id,
        )
        robot = pyb_utils.Robot(franka_id, client_id=client_id)
        for i in range(pyb.getNumJoints(franka_id)):
            joint_info = pyb.getJointInfo(franka_id, i)
            joint_name = joint_info[1].decode("utf-8")
            if joint_name == "panda_finger_joint1" or joint_name == "panda_finger_joint2":
                pyb.resetJointState(franka_id, i, 0.04, physicsClientId=client_id)
            
        # some cubes for obstacles
        cube1_id = pyb.loadURDF(
            "WallObstacle.urdf", useFixedBase=True, physicsClientId=client_id
        )
    
        obstacles = {
            "ground": ground_id,
            "cube1": cube1_id,
            "robot": franka_id
        }
        return robot, obstacles
    elif difficulty == 2:
        pyb.setTimeStep(TIMESTEP, physicsClientId=client_id)
        pyb.setAdditionalSearchPath(
            pybullet_data.getDataPath(), physicsClientId=client_id
        )
        
        # ground plane
        ground_id = pyb.loadURDF(
            "plane.urdf", [0, 0, 0], useFixedBase=True, physicsClientId=client_id
        )
    
        # KUKA iiwa robot arm (franka)
        franka_id = pyb.loadURDF(
            "franka.urdf",
            [0, 0, 0],
            useFixedBase=True,
            physicsClientId=client_id,
        )
        robot = pyb_utils.Robot(franka_id, client_id=client_id)
        for i in range(pyb.getNumJoints(franka_id)):
            joint_info = pyb.getJointInfo(franka_id, i)
            joint_name = joint_info[1].decode("utf-8")
            if joint_name == "panda_finger_joint1" or joint_name == "panda_finger_joint2":
                pyb.resetJointState(franka_id, i, 0.04, physicsClientId=client_id)
            
        # some cubes for obstacles
        cube1_id = pyb.loadURDF(
            "wall2.urdf", useFixedBase=True, physicsClientId=client_id
        )
    
        obstacles = {
            "ground": ground_id,
            "cube1": cube1_id,
            "robot": franka_id
        }
        return robot, obstacles
    
    elif difficulty == 3:
        pyb.setTimeStep(TIMESTEP, physicsClientId=client_id)
        pyb.setAdditionalSearchPath(
            pybullet_data.getDataPath(), physicsClientId=client_id
        )
        
        # ground plane
        ground_id = pyb.loadURDF(
            "plane.urdf", [0, 0, 0], useFixedBase=True, physicsClientId=client_id
        )
    
        # KUKA iiwa robot arm (franka)
        franka_id = pyb.loadURDF(
            "franka.urdf",
            [0, 0, 0],
            useFixedBase=True,
            physicsClientId=client_id,
        )
        robot = pyb_utils.Robot(franka_id, client_id=client_id)
        for i in range(pyb.getNumJoints(franka_id)):
            joint_info = pyb.getJointInfo(franka_id, i)
            joint_name = joint_info[1].decode("utf-8")
            if joint_name == "panda_finger_joint1" or joint_name == "panda_finger_joint2":
                pyb.resetJointState(franka_id, i, 0.04, physicsClientId=client_id)
            
        # some cubes for obstacles
        cube1_id = pyb.loadURDF(
            "wall3_l.urdf", useFixedBase=True, physicsClientId=client_id
        )
        cube2_id = pyb.loadURDF(
            "wall3_r.urdf", useFixedBase=True, physicsClientId=client_id
        )
        obstacles = {
            "ground": ground_id,
            "cube1": cube1_id,
            "cube2": cube2_id,
            "robot": franka_id
        }
        return robot, obstacles
    

def calculate_state_validity(current_joint_state):
    env_num = 2
    
    full_angles = franka_chain.active_to_full(current_joint_state, np.zeros(len(franka_chain.links)))
    current_fk = franka_chain.forward_kinematics(full_angles)
    x, y, z = current_fk[:3, 3]
    all_x.append(x)
    all_y.append(y)
    all_z.append(z)
    
    # Nur ein DIRECT-Client für Kollisionserkennung 
    col_id = pyb.connect(pyb.DIRECT)

    # Lade Umgebung
    col_robot, col_obstacles = load_environment(col_id, env_num)
    if env_num == 3:
        obstacles = [col_obstacles["ground"], col_obstacles["cube1"], col_obstacles["cube2"]]
    else:
        obstacles = [col_obstacles["ground"], col_obstacles["cube1"]]

    # Links des Roboters (Namen so wie im original)
    link = []
    link.append((col_robot.uid, "panda_panda_link1"))
    link.append((col_robot.uid, "panda_panda_link2"))
    link.append((col_robot.uid, "panda_panda_link3"))
    link.append((col_robot.uid, "panda_panda_link4"))
    link.append((col_robot.uid, "panda_panda_link5"))
    link.append((col_robot.uid, "panda_panda_link6"))
    link.append((col_robot.uid, "panda_panda_link7"))
    link.append((col_robot.uid, "panda_panda_hand"))
    link.append((col_robot.uid, "panda_panda_leftfinger"))
    link.append((col_robot.uid, "panda_panda_rightfinger"))
    collision_pairs = []
    for link_idx in link:
        for obs in obstacles:
            collision_pairs.append((obs, link_idx))

    col_detector = pyb_utils.CollisionDetector(
        col_id,
        collision_pairs
    )

    # Setze Gelenkzustand im kollisions-Client
    col_robot.reset_joint_configuration(current_joint_state)

    # Prüfe Kollision (Margin fest auf 0.01 gesetzt)
    if not col_detector.in_collision(margin=0.1):
        pyb.disconnect(col_id)
        print(f"State valid")
        return True
    else:
        pyb.disconnect(col_id)
        print(f"State invalid")
        return False


if __name__ == "__main__":
    current_state = [ 0.49749239, -0.39204911, -0.38585028, -1.32839171,  1.41997276,
  0.3847102,  -1.79493424]

    valid = calculate_state_validity(current_state)
    


pybullet build time: Jun 10 2025 11:09:21


State valid
b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
wall2

In [5]:
import sys
import time
try:
    from ompl import util as ou
    from ompl import base as ob
    from ompl import geometric as og
except ImportError:
    # if the ompl module is not in the PYTHONPATH assume it is installed in a
    # subdirectory of the parent directory called "py-bindings."
    from os.path import abspath, dirname, join
    sys.path.insert(0, join(dirname(dirname(abspath(__file__))), 'py-bindings'))
    from ompl import util as ou
    from ompl import base as ob
    from ompl import geometric as og
from math import sqrt
import argparse

j = 0
tested_joint_states = []
class ValidityChecker(ob.StateValidityChecker):
    def __init__(self, si, min_distance):
        super(ValidityChecker, self).__init__(si)
        self.min_distance = min_distance
    # Returns whether the given state's position overlaps the
    # circular obstacle
    
    def isValid(self, state):       
        joint_angles = [state[i] for i in range(7)]
        global j
        j = j+1
        #full_angles = franka_chain.active_to_full(joint_angles, np.zeros(len(franka_chain.links)))
        # current_fk = franka_chain.forward_kinematics(full_angles)
        # x, y, z = current_fk[:3, 3]
        # all_x.append(x)
        # all_y.append(y)
        # all_z.append(z)
        valid = calculate_state_validity(joint_angles)
        tested_joint_states.append(joint_angles[:])
        if not valid:
            print(f"Invalid state: {joint_angles}")
        return valid
        #return True

    # Returns the distance from the given state's position to the
    # boundary of the circular obstacle.
    def clearance(self, state):
        # Extract the robot's (x,y) position from its state
        print("hallo")
        x = state[0]
        y = state[1]
        z = state[2]

  
        

        # Distance formula between three points, offset by the circle's
        # radius
        return sqrt((x - 0.5)**2 + (y - 0.5)**2 + (z - 0.5)**2) - 0.05

    def checkCollisions(self, state):
        pass

## Returns a structure representing the optimization objective to use
#  for optimal motion planning. This method returns an objective
#  which attempts to minimize the length in configuration space of
#  computed paths.
def getPathLengthObjective(si):
    return ob.PathLengthOptimizationObjective(si)

## Returns an optimization objective which attempts to minimize path
#  length that is satisfied when a path of length shorter than 1.51
#  is found.
def getThresholdPathLengthObj(si):
    obj = ob.PathLengthOptimizationObjective(si)
    obj.setCostThreshold(ob.Cost(1.51))
    return obj

## Defines an optimization objective which attempts to steer the
#  robot away from obstacles. To formulate this objective as a
#  minimization of path cost, we can define the cost of a path as a
#  summation of the costs of each of the states along the path, where
#  each state cost is a function of that state's clearance from
#  obstacles.
#
#  The class StateCostIntegralObjective represents objectives as
#  summations of state costs, just like we require. All we need to do
#  then is inherit from that base class and define our specific state
#  cost function by overriding the stateCost() method.
#
class ClearanceObjective(ob.StateCostIntegralObjective):
    def __init__(self, si):
        super(ClearanceObjective, self).__init__(si, True)
        self.si_ = si

    # Our requirement is to maximize path clearance from obstacles,
    # but we want to represent the objective as a path cost
    # minimization. Therefore, we set each state's cost to be the
    # reciprocal of its clearance, so that as state clearance
    # increases, the state cost decreases.
    def stateCost(self, s):
        return ob.Cost(1 / (self.si_.getStateValidityChecker().clearance(s) +
                            sys.float_info.min))

## Return an optimization objective which attempts to steer the robot
#  away from obstacles.
def getClearanceObjective(si):
    return ClearanceObjective(si)

## Create an optimization objective which attempts to optimize both
#  path length and clearance. We do this by defining our individual
#  objectives, then adding them to a MultiOptimizationObjective
#  object. This results in an optimization objective where path cost
#  is equivalent to adding up each of the individual objectives' path
#  costs.
#
#  When adding objectives, we can also optionally specify each
#  objective's weighting factor to signify how important it is in
#  optimal planning. If no weight is specified, the weight defaults to
#  1.0.
def getBalancedObjective1(si):
    lengthObj = ob.PathLengthOptimizationObjective(si)
    clearObj = ClearanceObjective(si)

    opt = ob.MultiOptimizationObjective(si)
    opt.addObjective(lengthObj, 5.0)
    opt.addObjective(clearObj, 1.0)

    return opt

## Create an optimization objective equivalent to the one returned by
#  getBalancedObjective1(), but use an alternate syntax.
#  THIS DOESN'T WORK YET. THE OPERATORS SOMEHOW AREN'T EXPORTED BY Py++.
# def getBalancedObjective2(si):
#     lengthObj = ob.PathLengthOptimizationObjective(si)
#     clearObj = ClearanceObjective(si)
#
#     return 5.0*lengthObj + clearObj


## Create an optimization objective for minimizing path length, and
#  specify a cost-to-go heuristic suitable for this optimal planning
#  problem.
def getPathLengthObjWithCostToGo(si):
    obj = ob.PathLengthOptimizationObjective(si)
    obj.setCostToGoHeuristic(ob.CostToGoHeuristic(ob.goalRegionCostToGo))
    return obj


# Keep these in alphabetical order and all lower case
def allocatePlanner(si, plannerType):
    if plannerType.lower() == "bfmtstar":
        return og.BFMT(si)
    elif plannerType.lower() == "bitstar":
        return og.BITstar(si)
    elif plannerType.lower() == "fmtstar":
        return og.FMT(si)
    elif plannerType.lower() == "informedrrtstar":
        return og.InformedRRTstar(si)
    elif plannerType.lower() == "prmstar":
        return og.PRMstar(si)
    elif plannerType.lower() == "rrtstar":
        return og.RRTstar(si)
    elif plannerType.lower() == "sorrtstar":
        return og.SORRTstar(si)
    else:
        ou.OMPL_ERROR("Planner-type is not implemented in allocation function.")


# Keep these in alphabetical order and all lower case
def allocateObjective(si, objectiveType):
    if objectiveType.lower() == "pathclearance":
        return getClearanceObjective(si)
    elif objectiveType.lower() == "pathlength":
        return getPathLengthObjective(si)
    elif objectiveType.lower() == "thresholdpathlength":
        return getThresholdPathLengthObj(si)
    elif objectiveType.lower() == "weightedlengthandclearancecombo":
        return getBalancedObjective1(si)
    else:
        ou.OMPL_ERROR("Optimization-objective is not implemented in allocation function.")



def plan(runTime, plannerType, objectiveType, interpolationnumber, min_distance, start_xyz, goal_xyz, max_runtime, optimizingTime):

    # 1. Setup des StateSpace
    space = ob.RealVectorStateSpace(7)
    bounds = ob.RealVectorBounds(7)
    for i in range(7):
        bounds.setLow(i, joint_limits_rad_min[i])
        bounds.setHigh(i, joint_limits_rad_max[i])
    space.setBounds(bounds)

    # 2. Inverse Kinematics für Start
    prev_angle =  [1, -0.38066329, 0.26839613, -2, 0.77729231, 2, 0.00391651, 0.79768890]
    full_prev_angle = franka_chain.active_to_full(prev_angle, np.zeros(len(franka_chain.links)))
    
    start_frame = np.eye(4)
    start_frame[:3, 3] = start_xyz
    start_frame[:3, :3] = R.from_euler('xyz', [0, 0, -90], degrees=True).as_matrix()
    joints_start = franka_chain.inverse_kinematics_frame(start_frame, initial_position=full_prev_angle)
    
    if calculate_state_validity(joints_start) != True:
        print("recalculating start state")
        while joints_start != True:
            joints_start = franka_chain.inverse_kinematics_frame(start_frame, initial_position=full_prev_angle)
            calculate_state_validity(joints_start)
    else: 
        print("not recalculating start state")
            
    
    start = ob.State(space)
    for i in range(7):
        start[i] = joints_start[i+1]

    # 3. Inverse Kinematics für Goal
    target_frame = np.eye(4)
    target_frame[:3, 3] = goal_xyz
    target_frame[:3, :3] = R.from_euler('x', 180, degrees=True).as_matrix()
    joints_goal = franka_chain.inverse_kinematics_frame(target_frame, initial_position=full_prev_angle, orientation_mode="all")

    goal = ob.State(space)
    for i in range(7):
        goal[i] = joints_goal[i+1]

    fks=franka_chain.forward_kinematics(joints_start)
    fkg=franka_chain.forward_kinematics(joints_goal)
    print(joints_start, joints_goal)
    print(fks, fkg)
        
    # Construct a space information instance for this state space
    si = ob.SpaceInformation(space)

    # Set the object used to check which states in the space are valid
    validityChecker = ValidityChecker(si, min_distance)
    si.setStateValidityChecker(validityChecker)

    si.setup()
    

    # Create a problem instance
    pdef = ob.ProblemDefinition(si)

    # Set the start and goal states
    pdef.setStartAndGoalStates(start, goal)

    # Create the optimization objective specified by our command-line argument.
    # This helper function is simply a switch statement.
    pdef.setOptimizationObjective(allocateObjective(si, objectiveType))

    # Construct the optimal planner specified by our command line argument.
    # This helper function is simply a switch statement.
    optimizingPlanner = allocatePlanner(si, plannerType)

    # Set the problem instance for our planner to solve
    optimizingPlanner.setProblemDefinition(pdef)
    optimizingPlanner.setup()
    
    
    # attempt to solve the planning problem in the given runtime
    solved = optimizingPlanner.solve(runTime)
    start_time = time.time()
    while (str(solved) != "Exact solution"): 
        if time.time() - start_time >= max_runtime:
            print("Maximale runtime erreicht! Keine exakte Lösung gefunden.")
            break
        solved = optimizingPlanner.solve(runTime)
    
    if str(solved) == "Exact solution":
        finish_time = time.time() - start_time
        print("Exakte lösung gefunden in: ", finish_time, " Sekunden")
        first_path = pdef.getSolutionPath()
        first_path.interpolate(interpolationnumber)
        first_path_states = first_path.getStates()
        first_sol = [[s[i] for i in range(7)] for s in first_path_states]
        print(first_sol)
        np.save("first_sol.npy", first_sol)
        initial_cost = pdef.getSolutionPath().cost(pdef.getOptimizationObjective()).value()
        print('{0} found solution of path length {1:.4f} with an optimization ' \
            'objective value of {2:.4f}'.format( \
            optimizingPlanner.getName(), \
            pdef.getSolutionPath().length(), \
            initial_cost))

        #optimiting Path
        optimizingPlanner.solve(optimizingTime)
        path = pdef.getSolutionPath()
        path.interpolate(interpolationnumber)

        cost = pdef.getSolutionPath().cost(pdef.getOptimizationObjective()).value()
        print('{0} found solution of path length {1:.4f} with an optimization ' \
            'objective value of {2:.4f}'.format( \
            optimizingPlanner.getName(), \
            pdef.getSolutionPath().length(), \
            cost))
        
        path_states = path.getStates()
        opt_sol = [[s[i] for i in range(7)] for s in path_states]
    
        return opt_sol, initial_cost, cost, finish_time
    else:
        print("No solution found.")
        return (None, None, None, None)

if __name__ == "__main__":
    runtime = 2.0
    plannerType = "RRTstar"
    objectiveType = "PathLength"
    interpolationnumber = 20
    min_distance = 0.01
    max_runtime = 120 
    optimizingTime = 20
    
    # start_xyz = [0.1, 0.1, 0.85]
    # goal_xyz = [0.65, 0.3, 0.4]
    # points, cost = plan(runtime, plannerType, objectiveType, interpolationnumber, min_distance, start_xyz, goal_xyz, max_runtime, optimizingTime)
    # np.save("ompl_angles.npy", points)
    # print(points)
    # print(j)
    # np.save("tested_joint_states.npy", tested_joint_states)
## @endcond

In [6]:
def run_planner(run_id, plannerType, objectiveType, start_xyz, goal_xyz,
                interpolationnumber=20, min_distance=0.01, max_runtime=100, optimizingTime=5):
    
    points, initial_cost, cost, time = plan(
        runTime=2.0,
        plannerType=plannerType,
        objectiveType=objectiveType,
        interpolationnumber=interpolationnumber,
        min_distance=min_distance,
        start_xyz=start_xyz,
        goal_xyz=goal_xyz,
        max_runtime=max_runtime,
        optimizingTime=optimizingTime
    )
    
    return {
        "run_id": run_id,
        "planner": plannerType,
        "objective": objectiveType,
        "initial_cost": initial_cost,
        "cost": cost,
        "path": points,
        "time": time,
        "success": points is not None,
    }
#p = run_planner(1, "RRTstar", "PathLength", [0.1, 0.1, 0.85], [0.65, 0.3, 0.4])


In [ ]:
import os
import csv
results = []
repetitions = 1
output_dir = "ompl_results2"
os.makedirs(output_dir, exist_ok=True)

planner = "RRTstar"
objective = "PathLength"

# env 1 states
start1 = [0.1, 0.1, 0.85]
goal1 = [0.65, 0.3, 0.4]

# env 2 states
start2 = [0.4, -0.2, 0.2]
goal2 = [0.4, 0.3, 0.2]

# env 3 states
start3 = [0.3, -0.3, 0.15]
goal2 = [0.7, 0.3, 0.15]

for i in range(repetitions):
    print(f"Run {i+1}/{repetitions} for {planner} + {objective}")
    result = run_planner(
        run_id=i,
        plannerType=planner,
        objectiveType=objective,
        start_xyz=start2,
        goal_xyz=goal2
    )
    results.append(result)

summary_path = os.path.join(output_dir, f"run_summary_{planner}.csv")
file_exists = os.path.isfile(summary_path)
    
with open(summary_path, 'a', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["run_id", "planner", "objective", "initial_cost", "cost", "time", "path"])
    if not file_exists:
        writer.writeheader()
    for result in results:
        writer.writerow({
            "run_id": result["run_id"],
            "planner": result["planner"],
            "objective": result["objective"],
            "initial_cost": result["initial_cost"],
            "cost": result["cost"],
            "time": result["time"],
            "path": result["path"]
        })


Run 1/1 for RRTstar + PathLength
b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
wall2State valid
not recalculating start state
[ 0.          0.30707037  0.13643167 -0.52445283 -2.32175148 -0.66511634
  1.94042342  2.85099536  0.          0.          0.          0.        ] [ 0.          0.36372114  0.88337304  0.06282479 -1.18284467 -0.05513613
  2.06461312 -0.35562577  0.          0.          0.          0.        ]
[[-0.88169768  0.13457878 -0.45221427  0.39999999]
 [ 0.39138081  0.74389171 -0.54170673 -0.19999999]
 [ 0.26349622 -0.65460956 -0.70855915  0.20000002]
 [ 0.          0.          0.          1.        ]] [[ 1.00000000e+00  3.52932793e-13  1.76137839e-13  7.00000000e-01]
 [ 3.53219573e-13 -1.00000000e+00 -3.06539700e-14  3.00000000e-01]
 [ 1.75932747e-13  3.06868062e-14 -1.0000

         at line 249 in /mnt/c/Users/volle/Downloads/ompl-1.7.0/src/ompl/base/src/Planner.cpp
Error:   RRTstar: There are no valid initial states!
         at line 193 in /mnt/c/Users/volle/Downloads/ompl-1.7.0/src/ompl/geometric/planners/rrt/src/RRTstar.cpp
Error:   RRTstar: There are no valid initial states!
         at line 193 in /mnt/c/Users/volle/Downloads/ompl-1.7.0/src/ompl/geometric/planners/rrt/src/RRTstar.cpp
Error:   RRTstar: There are no valid initial states!
         at line 193 in /mnt/c/Users/volle/Downloads/ompl-1.7.0/src/ompl/geometric/planners/rrt/src/RRTstar.cpp
Error:   RRTstar: There are no valid initial states!
         at line 193 in /mnt/c/Users/volle/Downloads/ompl-1.7.0/src/ompl/geometric/planners/rrt/src/RRTstar.cpp
Error:   RRTstar: There are no valid initial states!
         at line 193 in /mnt/c/Users/volle/Downloads/ompl-1.7.0/src/ompl/geometric/planners/rrt/src/RRTstar.cpp
Error:   RRTstar: There are no valid initial states!
         at line 193 in /mnt

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd

rrt = pd.read_csv("ompl_results/run_summary_RRTstar.csv")

fig, axs = plt.subplots(1, 2, figsize=(12, 5))

plt.suptitle("RRT*", fontsize=16)
# Erster Plot: time
axs[0].plot(rrt["time"], marker='o', linestyle='-')
axs[0].axhline(rrt["time"].mean(), linestyle='--', color="red", label="Durchschnitt")
axs[0].set_title("Laufzeit (Time)")
axs[0].set_xlabel("Versuch")
axs[0].set_ylabel("Zeit in Sekunden")
axs[0].set_ylim(0, 100)

# Zweiter Plot: cost
axs[1].plot(rrt["initial_cost"], marker='o', linestyle='-')
axs[1].axhline(rrt["initial_cost"].mean(), linestyle='--', color="red", label="Durchschnitt")
axs[1].set_title("Kosten (Cost)")
axs[1].set_xlabel("Versuch")
axs[1].set_ylabel("Pfadkosten")
axs[1].set_ylim(0, 10)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os

# Liste deiner Algorithmus-Dateien und Namen
algorithms = {
    "RRTstar": "ompl_results/run_summary_RRTstar.csv",
    "PRMstar": "ompl_results/run_summary_prmstar.csv",
    "Informedrrtstar": "ompl_results/run_summary_informedrrtstar.csv"
}

num_algos = len(algorithms)
fig, axs = plt.subplots(num_algos, 2, figsize=(12, 5 * num_algos))

# Falls nur ein Algorithmus, axs in 2D-Array konvertieren
if num_algos == 1:
    axs = [axs]

for idx, (name, path) in enumerate(algorithms.items()):
    df = pd.read_csv(path)

    # Plot Time
    axs[idx][0].plot(df["time"], marker='o', linestyle='-')
    axs[idx][0].axhline(df["time"].mean(), linestyle='--', color="red", label="Durchschnitt")
    axs[idx][0].set_title(f"{name}: Laufzeit")
    axs[idx][0].set_xlabel("Versuch")
    axs[idx][0].set_ylabel("Zeit in Sekunden")
    axs[idx][0].set_ylim(0, 100)
    axs[idx][0].legend()

    # Plot Cost
    axs[idx][1].plot(df["cost"], marker='o', linestyle='-')
    axs[idx][1].axhline(df["cost"].mean(), linestyle='--', color="red", label="Durchschnitt")
    axs[idx][1].set_title(f"{name}: Pfadkosten")
    axs[idx][1].set_xlabel("Versuch")
    axs[idx][1].set_ylabel("Pfadkosten")
    axs[idx][1].set_ylim(0, 10)
    axs[idx][1].legend()

plt.tight_layout(rect=[0, 0, 1, 0.96])  # Platz für suptitle lassen
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np


def plot_xyz(x, y, z, start_xyz, goal_xyz, tested_points= None):
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(1, 3, 1, projection='3d')
    firx = fig.add_subplot(1, 3, 2, projection='3d')
    finx = fig.add_subplot(1, 3, 3, projection='3d')
    configure_3d_axes(ax)
    configure_3d_axes(firx)
    configure_3d_axes(finx)
    ax.set_title("Sampled Points")
    firx.set_title("First Trajectory")
    finx.set_title("Final Trajectory")

    start_xyz = np.array(start_xyz)
    goal_xyz = np.array(goal_xyz)

    first_traj= np.load("first_sol.npy")
    first_traj_full =[]
    first_traj_fk =[]
    for i in range(len(first_traj)):
        first_traj_full.append(franka_chain.active_to_full(first_traj[i], np.zeros(len(franka_chain.links))))
        first_traj_fk.append(franka_chain.forward_kinematics(first_traj_full[i]))

    first_sol = np.array([traj[0:3, 3] for traj in first_traj_fk])
    
    final_traj= np.load("ompl_angles.npy")
    full_traj =[]
    final_traj_fk =[]
    for i in range(len(final_traj)):
        full_traj.append(franka_chain.active_to_full(final_traj[i], np.zeros(len(franka_chain.links))))
        final_traj_fk.append(franka_chain.forward_kinematics(full_traj[i]))

    final_sol = np.array([traj[0:3, 3] for traj in final_traj_fk])


    
    # if tested_points is not None:
    #     ax.scatter(tested_points[0], tested_points[1], tested_points[2], c='gray', marker='.', alpha=0.3, label='Checked points')

    
    # Geplante Punkte + Start + Ziel
    ax.scatter(x, y, z, c='b', marker='o', label='Tested Points',zorder=1)
    ax.scatter(start_xyz[0], start_xyz[1], start_xyz[2], c='r', marker='o', label='Start')
    ax.scatter(goal_xyz[0], goal_xyz[1], goal_xyz[2], c='g', marker='o', label='Goal')

     # Erste gelöste Trajektorie + Start + Ziel
    firx.scatter(first_sol[:, 0], first_sol[:, 1], first_sol[:, 2], c='black', marker='o', label='First Trajectory', zorder=5)
    firx.scatter(start_xyz[0], start_xyz[1], start_xyz[2], c='r', marker='o', label='Start')
    firx.scatter(goal_xyz[0], goal_xyz[1], goal_xyz[2], c='g', marker='o', label='Goal')

    # Finale Trajektorie + Start + Ziel
    finx.scatter(final_sol[:, 0], final_sol[:, 1], final_sol[:, 2], c='black', marker='o', label='Final Trajectory', zorder=5)
    finx.scatter(start_xyz[0], start_xyz[1], start_xyz[2], c='r', marker='o', label='Start')
    finx.scatter(goal_xyz[0], goal_xyz[1], goal_xyz[2], c='g', marker='o', label='Goal')

    plt.show()
    print(len(x))

def configure_3d_axes(ax):
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    ax.set_zlim(-1, 1)
    ax.legend()
  
    
plot_xyz(all_x, all_y, all_z, [0.1, 0.1, 0.85], [0.65, 0.3, 0.4])

In [ ]:
states = np.load("tested_joint_states.npy")
print(len(states))


In [ ]:
import numpy as np

def load_waypoints():
    # Wegpunkte aus Datei laden
    waypoints = []
    with open("output.txt", "r") as f:
        for line in f:
            if line.strip():
                x, y, z = map(float, line.strip().split())
                waypoints.append(np.array([x, y, z]))
    return waypoints
    print(waypoints)
print(load_waypoints())

In [ ]:
def interpolate_trajectory(anglelist, steps_per_segment=20):
    interpolated = []
    for i in range(len(anglelist) - 1):
        start = np.array(anglelist[i])
        end = np.array(anglelist[i + 1])
        segment = np.linspace(start, end, steps_per_segment, endpoint=False)
        interpolated.extend(segment)
    interpolated.append(anglelist[-1])  # letzter Punkt anhängen
    return np.array(interpolated)


In [ ]:
# objekte in simulation/ andere objekte in pfadplanung
# mehr Pfadpunkte kalkulieren
# TCP nach unten gerichtet
# Kolision von Roboter vermeiden
# pick and place komplett modellieren
# statt txt export mit Api arbeiten
